In [2]:
import numpy as np
import pandas as pd

In [3]:
job_df = pd.read_csv("F:/Candidate_matching/data/recommendations.csv")

job_df.head(2)

,Unnamed: 0,Respondent,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,0,1,https://www.dice.com/jobs/detail/Full-Stack-De...,Saicon Consultants Inc.,"Contract W2, 12 Months",Proficient in RESTful web servicesProficient i...,Dice Id : saicon,"Pleasanton, CA",Full Stack Developer,10 hours ago,Telecommuting not available|Travel not required,NaN,Full Stack Developer,90d755eb5b34547c31978ce7a6eb519f
1,1,1,https://www.dice.com/jobs/detail/Full-Stack-Ja...,RedRiver Systems L.L.C.,"C2H W2, 3+ Months",Opportunity for an Open / Full Stack Java Engi...,Dice Id : 10122941,"Dallas, TX",Full Stack Java Engineer,14 hours ago,Telecommuting not available|Travel not required,www.dice.com,"Java, NodeJS, Ruby, Scala, Python, Powershell,...",4b072aab5c96491c2d7f6731772b8825


In [4]:
job_df.shape

(2000, 14)

In [5]:
for col in job_df.columns:
    print(col)

Unnamed: 0
Respondent
advertiserurl
company
employmenttype_jobstatus
jobdescription
jobid
joblocation_address
jobtitle
postdate
shift
site_name
skills
uniq_id


In [6]:
job_df = job_df[['uniq_id', 'jobtitle', 'company', 'jobdescription','advertiserurl']]
job_df.head(2)
job_df.shape

(2000, 5)

In [7]:
job_df['jobdescription'][20]

"The speed of technology. The influx of data. Shifting regulations. Increased customer expectations. That's where you'll find DST, helping clients find opportunity in ever-changing and complex customer, business, and regulatory requirements. Mastering complexity is a constantly changing and evolving challenge - one that only a few people can embrace and thrive upon. If you see your opportunity in complexity, then we are the right fit for you. Job Description: Organizational Overview Through its subsidiary companies, ALPS Holdings, Inc. is a leading provider of innovative investment products and customized servicing solutions to the financial services industry. Founded in 1985, Denver-based ALPS delivers its asset management and asset servicing solutions through offices in Boston, Dallas, Miami, New York, San Francisco, Seattle, and Toronto. ALPS is a wholly owned subsidiary of Kansas City-based DST Systems, Inc. For more information about ALPS and its services, visit www.alpsinc.com. I

In [8]:
job_df.isnull().sum()
job_df.fillna('',inplace=True)
job_df.isnull().sum()

uniq_id           0
jobtitle          0
company           0
jobdescription    0
advertiserurl     0
dtype: int64

In [9]:
job_df = job_df.drop_duplicates(subset=['uniq_id'])


In [10]:
job_df.shape

(757, 5)

In [11]:
job_df.describe()

,uniq_id,jobtitle,company,jobdescription,advertiserurl
count,757,757,757,757,757
unique,757,613,485,718,757
top,90d755eb5b34547c31978ce7a6eb519f,Full Stack Developer,CyberCoders,DescriptionHands-on Linux backend experience –...,https://www.dice.com/jobs/detail/Full-Stack-De...
freq,1,19,33,5,1


In [12]:
job_df['jobdescription'] = job_df['jobdescription'].str.lower()
job_df['jobtitle'] = job_df['jobtitle'].str.lower()

In [13]:
job_df['jobtitle'].unique()

array(['full stack developer', 'full stack java engineer',
       'full stack engineer', 'full-stack python developer',
       'full stack javascript engineer', 'python developer',
       'system administrator', 'application administrator',
       'java developer- full stack', 'senior full stack developer',
       'java full stack engineer (angular js is must)',
       'full stack software engineer', 'full stack .net developer',
       'full stack sde', '5 full stack engineers - long term engagements',
       '.net full stack developer', 'nms solution architect',
       'sr. systems test engineer (perm)',
       'oracle istore technical resource', 'sr. sql dba',
       'health care etl tester', 'etl production support', 'senior dba',
       'technology lead | telecom | oss/bss', 'oracle pl/sql', 'director',
       'software engineer (mountain view, w2)',
       'sr. front-end / angular web developer',
       'angular js full stack developer',
       'full time: senior .net developer in

In [17]:
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def tokenize_and_stem(text):
  """
  This function removes stopwords and stems the words in a text string.
  """
  stop_words = set(stopwords.words('english'))
  porter = PorterStemmer()
  text = text.lower()
  text = [porter.stem(word) for word in text.split() if word not in stop_words]
  return text

def cosine_similarity(doc1, doc2):
  """
  This function calculates the cosine similarity between two text documents.
  """
  intersection = set(doc1).intersection(set(doc2))
  magnitude1 = len(doc1) ** 0.5
  magnitude2 = len(doc2) ** 0.5
  if magnitude1 * magnitude2 == 0:
    return 0
  return len(intersection) / (magnitude1 * magnitude2)

def create_similarity_dictionary(job_titles):
  """
  This function creates a dictionary where the key is a job title and the value is a set of similar job titles based on cosine similarity. Sets ensure no duplicates.
  """
  similarity_dict = defaultdict(set)
  seen_pairs = set()  # Keep track of processed pairs to avoid duplicates
  for i, job_title in enumerate(job_titles):
    stemmed_job_title = tokenize_and_stem(job_title)
    for j, other_job_title in enumerate(job_titles[i+1:]):
      if i != j and (job_title, other_job_title) not in seen_pairs and (other_job_title, job_title) not in seen_pairs:
        seen_pairs.add((job_title, other_job_title))
        stemmed_other_job_title = tokenize_and_stem(other_job_title)
        similarity = cosine_similarity(stemmed_job_title, stemmed_other_job_title)
        if similarity > 0.7:  # You can adjust the threshold for similarity here
          similarity_dict[job_title].add(other_job_title)
          similarity_dict[other_job_title].add(job_title)
  return similarity_dict

# Example usage
job_titles = job_df['jobtitle']

similarity_dict = create_similarity_dictionary(job_titles)

print(similarity_dict)


defaultdict(<class 'set'>, {'full stack developer': {'c# angular full stack developer', '.net full stack developer', 'sr full stack security developer', 'full stack ui developers', 'senior full stack developer', 'full stack .net developer', 'full stack developer', 'walkin for full stack developer', 'position- full stack developer', 'full stack c#/.net developer - nosql', 'full stack c#/.net developer - w2 only', 'angular js full stack developer', 'sr. full stack developer', 'full stack, mean stack developer', 'lead full stack developer', 'full stack java developer'}, 'senior full stack developer': {'.net full stack developer', 'full stack ui developers', 'senior full stack developer', 'full stack .net developer', 'full stack developer', 'walkin for full stack developer', 'position- full stack developer', 'sr. full stack developer', 'lead full stack developer', 'full stack java developer', 'senior full stack engineer'}, 'full stack .net developer': {'.net full stack developer', 'full st

In [15]:
# job_df.to_csv('cleaned_recommendation.csv', index=False)